In [ ]:
import json, os, random, io, pdb
from tqdm import tqdm
from random import random
import numpy as np
import glob 
import pickle as pkl 
import re 

In [ ]:
test_loc = 'data/APPS/test/'
problems = glob.glob(test_loc + '*')
problems = sorted(problems) # Pin some ordering

In [ ]:
def find_in_out(lines):
    start_example = False
    start_input = False
    start_output = False 
    inputs = []
    outputs = []
    curr_input = ''
    curr_output = ''
    for line in lines:
        
        if len(line.strip())==0: 
            start_output = False
            start_input = False 
            continue
        
        line1 = line.lower()
        
        if '-examples-' in line1 or '-example-' in line1 or '-example -' in line1 or \
            '-example 1-' in line1 or '-example 2-' in line1 or '-example 3-' in line1 or \
            '-example 4-' in line1 or '-example 5-' in line1 or \
            'example:' in line1 or \
            'example 1:' in line1 or 'example 2:' in line1 or '-example 3:' in line1 or \
            'example 4:' in line1 or 'example 5:' in line1:
            start_example = True
            continue
        
        if '-note-'.lower() in line1:
            start_example = False
            start_output = False
            start_input = False
            continue
            
        if (start_example and 'Input' in line) or ('-Sample Input' in line) \
            or ('-Example Input' in line) or ('Sample Input:' in line) \
            or ('-Sample input' in line):
            start_input = True
            start_output = False
            
            if len(curr_output)>0:
                outputs.append(curr_output)
                curr_output = ''
            
            if (not '-sample input' in line1) and (not '-example input' in line1) and (not '-sample input' in line1):
                
                if 'input:' in line1:
                    temp = line1.replace('example','').replace('sample','').replace('input:','')
                    if len(temp.strip())>0: 
                        curr_input = temp        
            continue
        
        if (start_example and 'Output' in line) or ('-Sample Output' in line) \
            or ('-Example Output' in line) or ('Sample Output:' in line) \
            or ('-Sample output' in line):
            start_output = True
            start_input = False
            
            if len(curr_input)>0:
                inputs.append(curr_input)
                curr_input = ''
                
            if (not '-sample output' in line1) and (not '-example output' in line1) and (not '-sample output' in line1):
                if 'output:' in line1:
                    temp = line1.replace('example','').replace('sample','').replace('output:','')
                    if len(temp.strip())>0: 
                        curr_output = temp
            continue 
        
        if start_input:
            curr_input += line 
        
        if start_output:
            curr_output += line 
            
    if len(curr_output)>0: 
        outputs.append(curr_output)
        start_output = False
            
    if len(inputs)==0 or len(inputs) != len(outputs) or (start_output or start_input):
        return None
        
    return {'inputs': inputs, 'outputs': outputs}

In [ ]:
non_eng = {}
no_test = {}
in_outs = {}

for problem_idx, problem in tqdm(enumerate(problems),total=len(problems)): 
    problem_idx += start_idx
    prompt_path = os.path.join(problem, "question.txt")
    with open(prompt_path, 'r') as file:
        lines = file.readlines()
        temp = ''.join(lines)

    in_out = find_in_out(lines)
    in_outs[problem_idx] = in_out
    
    # special case with non-English problems
    if 'Входные' in temp: 
        non_eng[problem_idx]=temp
    
    elif in_outs[problem_idx] is None:
        no_test[problem_idx] =temp
    
    example_test = os.path.join(problem, "example_input_output.json")
    json.dump(in_out, open(example_test, 'w'))
    
print("Non-English task: {}".format(len(non_eng))) 
print("Zero-example-test task: {}".format(len(no_test)))       

'''
Non-English task: 17
Zero-example-test task: 46
'''

In [ ]:
nb_example_tests = []
for k,v in in_outs.items(): 
    if v is None:
        nb_example_tests.append(0)
    else:
        nb_example_tests.append(len(v['inputs']))

print("Total number of samples: {}".format(np.array(nb_example_tests).shape))
print("Average extracted example test cases: {}".format(np.array(nb_example_tests).mean()))

'''
Total number of samples: (5000,)
Average extracted example test cases: 1.9764
'''